In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request

import urllib.parse as p
import re
import os
import pickle

SCOPES = ["https://www.googleapis.com/auth/youtube.force-ssl"]

In [2]:
def youtube_authenticate():
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"
    api_service_name = "youtube"
    api_version = "v3"
    client_secrets_file = "credentials.json"
    creds = None
    # the file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first time
    if os.path.exists("token.pickle"):
        with open("token.pickle", "rb") as token:
            creds = pickle.load(token)
    # if there are no (valid) credentials availablle, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(client_secrets_file, SCOPES)
            creds = flow.run_local_server(port=0)
        # save the credentials for the next run
        with open("token.pickle", "wb") as token:
            pickle.dump(creds, token)

    return build(api_service_name, api_version, credentials=creds)

In [3]:
youtube = youtube_authenticate()

In [4]:
def get_video_id_by_url(url):
    """
    Return the Video ID from the video `url`
    """
    # split URL parts
    parsed_url = p.urlparse(url)
    # get the video ID by parsing the query of the URL
    video_id = p.parse_qs(parsed_url.query).get("v")
    if video_id:
        return video_id[0]
    else:
        raise Exception(f"Wasn't able to parse video URL: {url}")

In [5]:
def get_video_details(youtube, **kwargs):
    return youtube.videos().list(
        part="snippet,contentDetails,statistics",
        **kwargs
    ).execute()

In [45]:
def get_video_metadata(url):
    dat=get_video_details(youtube,id=get_video_id_by_url(url))["items"][0]
    
    res={}
    res["views"]=dat["statistics"]["viewCount"]
    res["likes"]=dat["statistics"]["likeCount"]
    res["dislikes"]=dat["statistics"]["dislikeCount"]
    res["comments"]=dat["statistics"]["commentCount"]
    
    res["title"]=dat["snippet"]["title"]
    res["desc"]=dat["snippet"]["description"]
    res["thumbnail"]=dat["snippet"]["thumbnails"]["maxres"]["url"]
    
    return res

In [46]:
get_video_metadata("https://www.youtube.com/watch?v=951X1IgtL5M")

{'views': '531135',
 'likes': '47327',
 'dislikes': '222',
 'comments': '1288',
 'title': 'How Skrillex makes his basslines',
 'desc': 'with a Tibetan monk in his computer..',
 'thumbnail': 'https://i.ytimg.com/vi/951X1IgtL5M/maxresdefault.jpg'}

In [8]:
def get_channel_videos(youtube, **kwargs):
    return youtube.search().list(
        **kwargs
    ).execute()

In [57]:
def crawl_channel(channelId):
    ids=[]
    nxt=None
    
    while(True):
        res=get_channel_videos(youtube,part="snippet",channelId=channelId,type="video",pageToken=nxt)
        videos=res.get("items")
        
        for video in videos:
            ids.append(video["id"]["videoId"])
            
        if "nextPageToken" in res:
            nxt=res["nextPageToken"]
        else:
            break
    
    return ids

In [58]:
crawl_channel("UCRWOdwLRsenx2jLaiCAIU4A")

['kmte4HGJCdo',
 'T71vkqZeniU',
 'R4zha-fcxH8',
 'E4kRDlsMLNM',
 'KhUYTvpNDNI',
 'ABp2hdvgkxk',
 'zTpu9bQmBlQ',
 'b-WtGGawa7M',
 '4xLWjHNrIys',
 'WV-1LGbCOG8',
 'AtLU2bHiVbE',
 'xbxP-s_-arc',
 'GwnGVNuuYWw',
 'pKE7iMKJGHg',
 'SxBrZ6tpgOQ',
 '54cchVisr2A',
 'KgRZ5ohwLKc',
 'YTyx9bXmMIw',
 'imkKlp3YadY',
 '_Lcstg3DVfg',
 'M3qjF2vGcA8',
 '5i_oD45sAWE',
 'tULklvbdi5w',
 'a1ENF43IIeI',
 'uDyNjSQVL6M',
 'TlMg3plBhiU',
 '39_AInFOrm8',
 '84d9ooGeH48',
 '4t_hUQNkwVU',
 'ANwXRqBq5ik',
 'TwX8tiuowU0',
 'X13cOgv1fxQ',
 'YTmvCBftDXk',
 'gGxyb_WlFmM',
 '9FYyxXpI9CM',
 'CLaiky2Cy4k',
 'e_NT_9fr7uE',
 'qt0InK2pofo',
 'J_f-Jc8sMw8',
 'lVE_CLfBwxY',
 'qopkBKmmGkE',
 'E9L7z5xu7Q4',
 'lXqVu6soRCQ',
 'WARGstLj150',
 'hNbQB2pHYnk',
 '5aAUbn7W6sg',
 'OnsIBD8h91A',
 'EF2DlcV0dUg',
 'fW6Kz4C6-TA',
 'vL2F55kMbyA',
 'XQfTgvhQeLw',
 'VndkM923RM8',
 'pYZ4AKs8XtQ',
 'uwhfH_ajsiE',
 'XoNlPHsUfTA',
 'GxFwNLfpdDQ',
 'f4MdGGPHu6w',
 '-080QH5__LM',
 '-Y87rT2z5XE',
 '9GzM-WXNEEc',
 'WHSNkzZtQdQ',
 'rylXYMJLs7A',
 'eOC0sI

In [61]:
crawl_channel("UCc7_woMAIVIW2mAr1rPCsFQ")

HttpError: <HttpError 403 when requesting https://youtube.googleapis.com/youtube/v3/search?part=snippet&channelId=UCc7_woMAIVIW2mAr1rPCsFQ&type=video&pageToken=CAUQAA&alt=json returned "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.". Details: "The request cannot be completed because you have exceeded your <a href="/youtube/v3/getting-started#quota">quota</a>.">

References

https://www.thepythoncode.com/article/using-youtube-api-in-python